In [17]:
import matplotlib.pyplot as plt
import networkx as nx
import sys
import pickle as pkl
import numpy as np
import hdbscan
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys
from functools import reduce
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from tqdm import tqdm
from copy import deepcopy
from pprint import pprint

def parse_index_file(filename):
    """Parse index file."""
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def sample_mask(idx, l):
    """Create mask."""
    mask = np.zeros(l)
    mask[idx] = 1
    return np.array(mask, dtype=np.bool)


def load_data(dataset_str):
    """
    Loads input data from gcn/data directory
    ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
        (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
    ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
    ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
    ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
    ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
        object;
    ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.
    All objects above must be saved using python pickle module.
    :param dataset_str: Dataset name
    :return: All data input files loaded (as well the training/test data).
    """
    names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
    objects = []
    for i in range(len(names)):
        with open("datasets/data-gcn/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
            if sys.version_info > (3, 0):
                objects.append(pkl.load(f, encoding='latin1'))
            else:
                objects.append(pkl.load(f))

    x, y, tx, ty, allx, ally, graph = tuple(objects)
    test_idx_reorder = parse_index_file("datasets/data-gcn/ind.{}.test.index".format(dataset_str))
    test_idx_range = np.sort(test_idx_reorder)

    if dataset_str == 'citeseer':
        # Fix citeseer dataset (there are some isolated nodes in the graph)
        # Find isolated nodes, add them as zero-vecs into the right position
        test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
        tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
        tx_extended[test_idx_range-min(test_idx_range), :] = tx
        tx = tx_extended
        ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
        ty_extended[test_idx_range-min(test_idx_range), :] = ty
        ty = ty_extended

    features = sp.vstack((allx, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

    labels = np.vstack((ally, ty))

    return adj, features, labels

def data_helper(dname):
    def sub_helper(G, keyname):
        labels = [G.nodes[i][keyname] for i in G.nodes]
        sorted_labels_set = sorted(list(set(labels)))
        labels_idx = np.array(list(map(lambda x: sorted_labels_set.index(x), labels)))
        return G, labels_idx
    if dname in ['cora', 'citeseer', 'pubmed']:
        adj, features, labels =load_data(dname)
        labels_idx = np.argmax(labels, axis=-1)
        G=nx.from_scipy_sparse_matrix(adj)
        return G, labels_idx
    elif dname == 'karate':
        G=nx.karate_club_graph()
        return sub_helper(G, 'club')
    elif dname == 'strike':
        G=nx.read_gml("datasets/real-classic/strike.gml")
        return sub_helper(G, 'value')
    elif dname == 'polbooks':
        G=nx.read_gml("datasets/real-classic/polbooks.gml")
        return sub_helper(G, 'value')
    elif 'lfr' in dname:
        mu = .1 * int(dname.split('_')[1])
        assert mu <= .9 and mu >= .1
        G = nx.algorithms.community.community_generators.LFR_benchmark_graph(
            1000, 3, 1.5, mu, average_degree=5, min_degree=None, max_degree=None, 
            min_community=20, max_community=None, tol=1e-07, max_iters=500, seed=None)
        for i in G:
            G.nodes[i]['community_tuple'] = tuple(sorted(list( G.nodes[i]['community'])))
        return sub_helper(G, 'community_tuple')
    
def partition2label(G, partition):
    c = 0
    node2idx = {}
    for i in G.nodes:
        node2idx[i] = c
        c += 1
    inferred_labels = np.zeros((reduce(lambda x, y: x + y, list(map(lambda z: len(z), partition)))))
    assert len(G) == len(inferred_labels)
    for ind, i in enumerate(partition):
        for j in i:
            inferred_labels[node2idx[j]] = ind
    return inferred_labels

In [2]:
all_graphs = {
    'lfr_mu_1': lambda: nx.algorithms.community.community_generators.LFR_benchmark_graph(
        1000, 3, 1.5, 0.1, average_degree=5, min_degree=None, max_degree=None, min_community=20, 
        max_community=None, tol=1e-07, max_iters=500, seed=None),
    'lfr_mu_5': lambda: nx.algorithms.community.community_generators.LFR_benchmark_graph(
        1000, 3, 1.5, 0.5, average_degree=5, min_degree=None, max_degree=None, min_community=20, 
        max_community=None, tol=1e-07, max_iters=500, seed=None),
    'lfr_mu_9': lambda: nx.algorithms.community.community_generators.LFR_benchmark_graph(
        1000, 3, 1.5, 0.9, average_degree=5, min_degree=None, max_degree=None, min_community=20, 
        max_community=None, tol=1e-07, max_iters=500, seed=None),
    'karate': lambda: nx.karate_club_graph(),
    'strike': lambda: nx.read_gml("datasets/real-classic/strike.gml"),
    'football': lambda: nx.read_gml("datasets/real-classic/football.gml"),
    'polbooks': lambda: nx.read_gml("datasets/real-classic/polbooks.gml"),
    
}

In [20]:
performance = {}
for i in ['cora', 'citeseer', 'pubmed', 'karate', 'strike', 'polbooks', 'lfr_3', 'lfr_6', 'lfr_9']:
    performance[i] = {
        'label_prop': {
            'nmi': [],
            'ari': [],
        },
        'svd': {
            'nmi': [],
            'ari': []
        }
    }

for dname in tqdm(performance):
    print('=====', dname)
    G, labels = data_helper(dname)
    for algo in performance[dname]:
        if 'lfr' in dname:
            runs = 10
        else:
            runs = 3
        for run in range(runs):
            if algo == 'label_prop':
                partition =  list(nx.community.label_propagation_communities(G))
                labels_inf = partition2label(G, partition)
            elif algo == 'svd':
                aja = nx.to_scipy_sparse_matrix(G)
                emb = TruncatedSVD(n_components=min(100, aja.shape[0] // 3)).fit_transform(aja)
                clusterer = hdbscan.HDBSCAN()
                clusterer.fit(emb)
                labels_inf = clusterer.labels_
            else:
                raise NotImplementedError()
            for metric in performance[dname][algo]:
                if metric == 'nmi':
                    performance[dname][algo][metric].append(normalized_mutual_info_score(labels, labels_inf))
                elif metric == 'ari':
                    performance[dname][algo][metric].append(adjusted_rand_score(labels, labels_inf))

  0%|                                                                                            | 0/9 [00:00<?, ?it/s]

===== cora


C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will u

===== citeseer


C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
 22%|██████████████████▋                                                                 | 2/9 [00:12<00:40,  5.72s/it]

...

C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will u

===== lfr_6


C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will u

===== lfr_9


C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
C:\Users\Junhao\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will u

In [40]:
performance_ = deepcopy(performance)
for dname in performance:
    for algo in performance[dname]:
        for metric in performance[dname][algo]:
            performance_[dname][algo][metric + '_mean'] = np.mean(performance[dname][algo][metric])
            performance_[dname][algo][metric + '_std'] = np.std(performance[dname][algo][metric])
performance__ = {}
for algo in performance[dname]:
    performance__[algo] = {'nmi': {}, 'ari': {}}
    temp_mean = []
    temp_std = []
    for dname in performance:
        for metric in performance[dname][algo]:
            performance__[algo][metric][dname] = (performance_[dname][algo][metric + '_mean'],
                                         performance_[dname][algo][metric + '_std'])
            temp_mean.append(performance_[dname][algo][metric + '_mean'])
            temp_std.append(performance_[dname][algo][metric + '_std'])

performance___ = {}
for algo in performance__:
    temp = []
    performance___[algo] = {}
    for metric in performance__[algo]:
        for dname in performance__[algo][metric]: 
            temp.append(performance__[algo][metric][dname][0])
        performance___[algo][metric] = np.mean(temp)

In [41]:
pprint(performance___)

{'label_prop': {'ari': 0.3059852145169937, 'nmi': 0.4296456856626334},
 'svd': {'ari': 0.10917536742373199, 'nmi': 0.19500130794934986}}
